### **Setup**

In [1]:
%%bash
pip install timm -q

### **Library Imports**

In [2]:
import os
import re
import cv2
import json
import timm
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from time import time
from typing import Union
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader as DL
from torchvision import transforms

from sklearn.metrics import accuracy_score

### **Utilities and Helpers**

In [3]:
SEED: int = 42
SIZE: int = 224
LABELS = json.load(open("/kaggle/input/db-dataframe/labels.json", "r"))    


def breaker(num: int=50, char: str="*") -> None: print("\n" + num*char + "\n")

    
def get_image(path: str, size: int=224) -> np.ndarray:
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(src=image, code=cv2.COLOR_BGR2RGB)
    if image.shape == (224, 224, 3): return image
    return cv2.resize(src=image, dsize=(size, size), interpolation=cv2.INTER_AREA)



### **Configuration**

In [4]:
class CFG(object):
    def __init__(self, 
                 seed: int = 42,
                 size: int = 224,
                 n_splits: int = 5,
                 batch_size: int = 16,
                 epochs: int = 25,
                 early_stopping: int = 5,
                 lr: float = 1e-4,
                 wd: float = 0.0,
                 max_lr: float = 1e-3,
                 pct_start: float = 0.2,
                 steps_per_epoch: int = 100,
                 div_factor: int = 1e3, 
                 final_div_factor: float = 1e3,
                 ):
        self.seed = seed
        self.size = size
        self.n_splits = n_splits
        self.batch_size = batch_size
        self.epochs = epochs
        self.early_stopping = early_stopping
        self.lr = lr
        self.wd = wd
        self.max_lr = max_lr
        self.pct_start = pct_start
        self.steps_per_epoch = steps_per_epoch
        self.div_factor = div_factor
        self.final_div_factor = final_div_factor
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        if self.size == 224:
            self.train_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([0.52618, 0.48953, 0.42159], [0.23276, 0.23118, 0.23232]),
                transforms.RandomAffine(degrees=(-45, 45), translate=(0.15, 0.15), scale=(0.5, 1.5)),
                transforms.RandomHorizontalFlip(p=0.25),
                transforms.RandomVerticalFlip(p=0.25),
            ])
            self.valid_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([0.52618, 0.48953, 0.42159], [0.23276, 0.23118, 0.23232]),
            ])
        
        if self.size == 384:
            self.train_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([0.52618, 0.48953, 0.42159], [0.23276, 0.23118, 0.23232]),
                transforms.RandomAffine(degrees=(-45, 45), translate=(0.15, 0.15), scale=(0.5, 1.5)),
                transforms.RandomHorizontalFlip(p=0.25),
                transforms.RandomVerticalFlip(p=0.25),
            ])
            self.valid_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([0.52618, 0.48953, 0.42159], [0.23276, 0.23118, 0.23232]),
            ])
        
        if self.size == 512:
            self.train_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([0.52618, 0.48953, 0.42159], [0.23276, 0.23118, 0.23232]),
                transforms.RandomAffine(degrees=(-45, 45), translate=(0.15, 0.15), scale=(0.5, 1.5)),
                transforms.RandomHorizontalFlip(p=0.25),
                transforms.RandomVerticalFlip(p=0.25),
            ])
            self.valid_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([0.52618, 0.48953, 0.42159], [0.23276, 0.23118, 0.23232]),
            ])
                                
        self.save_path = "saves"
        if not os.path.exists(self.save_path): os.makedirs(self.save_path)
    
cfg = CFG(
    seed=SEED, 
    size=SIZE
)

### **Dataset Template**

In [5]:
class DS(Dataset):
    def __init__(
        self, 
        filepaths: np.ndarray, 
        size: int,
        labels: Union[np.ndarray, None]=None, 
        transform=None
    ):
        
        self.filepaths = filepaths
        self.labels = labels
        self.size = size
        self.transform = transform
    
    def __len__(self):
        return self.filepaths.shape[0]
    
    def __getitem__(self, idx):
        image = get_image(self.filepaths[idx], self.size)
        if self.labels is None:
            return self.transform(image)
        return self.transform(image), torch.LongTensor(self.labels[idx])

### **Model**

In [6]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
            
        self.model = timm.create_model(model_name="efficientnet_b4", pretrained=False)
        self.model.classifier = nn.Linear(in_features=self.model.classifier.in_features, out_features=70)

    def forward(self, x):
        return nn.LogSoftmax(dim=1)(self.model(x))

### **Predict Helper**

In [7]:
def predict_batch(model=None, dataloader=None, path=None, device=None) -> np.ndarray:
    model.load_state_dict(torch.load(path, map_location=device)["model_state_dict"])
    model.to(device)    
    model.eval()
    
    y_pred = torch.zeros(1, 1).to(device)
    
    for X in dataloader:
        X = X.to(device)
        with torch.no_grad():
            output = torch.argmax(torch.exp(model(X)), dim=1)
        y_pred = torch.cat((y_pred, output.view(-1, 1)), dim=0)
    
    return y_pred[1:].detach().cpu().numpy().squeeze().astype("uint8")

### **Test**

In [8]:
ts_df = pd.read_csv("/kaggle/input/db-dataframe/test.csv")
filepaths = ts_df.filepaths.copy().values

y_true: list = []
for filepath in filepaths:
    y_true.append(filepath.split("/")[-2])
y_true = np.array(y_true)

ts_data_setup = DS(
    filepaths=filepaths,
    size=cfg.size,
    labels=None,
    transform=cfg.valid_transform
)
ts_data = DL(ts_data_setup, batch_size=64, shuffle=False)

model = Model().to(cfg.device)

y_pred = predict_batch(
    model=model,
    dataloader=ts_data,
    path="/kaggle/input/db-en4-a224-e10-f3/saves/ble_state_fold_3.pt",
    device=cfg.device
)
y_pred = np.array([LABELS[str(pred)] for pred in y_pred])

breaker()
print(f"Accuracy : {accuracy_score(y_true, y_pred) * 100:.2f}")
breaker()


**************************************************

Accuracy : 96.00

**************************************************

